In [3]:
import numpy as np 
import pandas as pd 
import os
import gc
import psutil

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model

from keras import optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
import keras

In [2]:
def RNN_model():

    ##Inputs
    #text
    title = Input(shape=[16], name="title")
    description = Input(shape=[128], name="description")
    #category        
    parent_category_name=Input(shape=[1], name="parent_category_name")
    category_name=Input(shape=[1], name="category_name")
    param_1 = Input(shape=[1], name="param_1")
    param123 = Input(shape=[1], name="param123")
    image_top_1 = Input(shape=[1], name="image_top_1")
    cluster_5 = Input(shape=[1], name="cluster_5")
    cluster_10 = Input(shape=[1], name="cluster_10")
    cluster_20 = Input(shape=[1], name="cluster_20")
    user_type = Input(shape=[1], name="user_type")
    activation_date = Input(shape=[1], name="activation_date")
    item_seq=Input(shape=[1], name="item_seq")
    #numerical
    #'item_seq_count','item_seq_deal_avg',
    #'dis_days', 'edit_time', 'item_num','count',      
    numerical= Input(shape=[6], name="numerical")
    #image
    #'avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
    #'avg_pixel_width', 'width', 'height', 'size', 
    #'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2', 
    #'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2', 
    #'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2'
    image= Input(shape=[25], name="image")
    #location
    #'cus', 'ship', 'r_cus', 'r_ship', 'all_cus'
    location=Input(shape=[5], name="location")
    price= Input(shape=[1], name="price")
    #prenum
    #'para_num',
    #'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
    #'title_num_words','title_num_unique_words','title_words_vs_unique'
    prenum= Input(shape=[8], name="prenum")
    #postnum
    #'description_num_words','description_num_unique_words','description_words_vs_unique',
    postnum= Input(shape=[3], name="postnum")
    
    #Embeddings layers
    vocab_size=200001
    EMBEDDING_DIM1=300
    emb_title = Embedding(vocab_size, EMBEDDING_DIM1, weights = [embedding_matrix1], 
                                          trainable = False)(title)
    emb_description = Embedding(vocab_size, EMBEDDING_DIM1, weights = [embedding_matrix1], 
                                          trainable = False)(description)
    emb_category_name = Embedding(48, 3)(category_name)
    emb_parent_category_name = Embedding(10, 3)(parent_category_name)
    emb_param_1 = Embedding(373, 1)(param_1)
    emb_param123 = Embedding(2402, 1)(param123)
    emb_image_top_1 = Embedding(3065, 1)(image_top_1)    
    emb_cluster_5 = Embedding(68, 1)(cluster_5)
    emb_cluster_10 = Embedding(47, 1)(cluster_10)
    emb_cluster_20 = Embedding(24, 1)(cluster_20)
    emb_user_type = Embedding(4, 1)(user_type)
    emb_activation_date = Embedding(8, 1)(activation_date)
    emb_item_seq = Embedding(33947, 1)(item_seq)
       

    rnn_layer1 = GRU(32) (emb_title)
    rnn_layer2 = GRU(16) (emb_description)
    
    inter_l=concatenate([rnn_layer1,
                         Flatten() (emb_category_name), 
                         Flatten() (emb_parent_category_name),
                         Flatten() (emb_cluster_5), 
                         Flatten() (emb_cluster_10), 
                         Flatten() (emb_cluster_20),
                         Flatten() (emb_param_1), 
                         Flatten() (emb_param123), 
                         location,
                         image,
                         prenum,
                         price])
    inter_l = Dense(16,activation='relu') (inter_l)
    
    post_l=concatenate([rnn_layer2,                        
                        image,
                        postnum,
                        price])
    post_l = Dense(16,activation='relu') (post_l)
    
    
    #main layer
    main_l = concatenate([inter_l,
                          post_l,                         
                          Flatten() (emb_image_top_1),                            
                          Flatten() (emb_user_type), 
                          Flatten() (emb_activation_date), 
                          Flatten() (emb_item_seq),                         
                          location,
                          numerical,
                          price])
    
    
    main_l = Dropout(0.0)(Dense(64,activation='relu') (main_l))
    main_l = Dropout(0.0)(Dense(64,activation='relu') (main_l))
    main_l = Dropout(0.0)(Dense(32,activation='relu') (main_l))
    main_l = Dropout(0.0)(Dense(16,activation='relu') (main_l))
    main_l = Dropout(0.0)(Dense(4,activation='relu') (main_l))
    
    #output
    output = Dense(1,activation="sigmoid") (main_l)
    
    #model
    
    model = Model(inputs=[title, description, parent_category_name, category_name, param_1, param123, image_top_1, 
                         cluster_5, cluster_10, cluster_20, user_type, activation_date, item_seq,
                          numerical,image,location,price,prenum,postnum],
                  outputs=[output,inter_l,post_l])
    model.compile(optimizer = 'adam', loss= root_mean_squared_error, metrics = [root_mean_squared_error],loss_weights=[1.0,0.0,0.0])
    return model

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred))) 

In [3]:
print(os.listdir('E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'))

['best_public_blend.csv', 'embedding_matrix1.npy', 'lgsub_noCV.csv', 'resultwithwhole.001.csv', 'RNNtest.csv', 'RNNtrain.csv', 'testfinal.csv', 'testready.csv', 'testtemp.csv', 'testx.csv', 'trainfinal.csv', 'trainready.csv', 'traintemp.csv', 'trainx.csv']


In [4]:

def get_keras_data(data1,data2,data3):
    title=[]
    for i in list(data1.title):
        title.append(list(eval(i)))
    description=[]
    for i in list(data1.description):
        description.append(list(eval(i)))
    
    title=pad_sequences(title, maxlen=16)
    description=pad_sequences(description, maxlen=128)
    parent_category_name=np.array(data2.parent_category_name)
    category_name=np.array(data2.category_name)
    param_1=np.array(data2.param_1)
    param123=np.array(data2.param123)
    image_top_1=np.array(data2.image_top_1)
    cluster_5=np.array(data2.cluster_5)
    cluster_10=np.array(data2.cluster_10)
    cluster_20=np.array(data2.cluster_20)
    user_type=np.array(data2.user_type)
    activation_date=np.array(data2.activation_date)
    item_seq=np.array(data2.item_seq_number)
    
    columns=['item_seq_count','item_seq_deal_avg','dis_days', 'edit_time', 'item_num','count']
    numerical=np.array(data3[columns])
    columns=['avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
             'avg_pixel_width', 'width', 'height', 'size',
             'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2',
             'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2',
             'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2']
    image=np.array(data3[columns])
    columns=['cus', 'ship', 'r_cus', 'r_ship', 'all_cus']
    location=np.array(data3[columns])
    price=np.array(data3.price)
    columns=['para_num',
             'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
             'title_num_words','title_num_unique_words','title_words_vs_unique']
    prenum=np.array(data3[columns])
    columns=['description_num_words','description_num_unique_words','description_words_vs_unique']
    postnum=np.array(data3[columns])
    
    output=np.array(data2.deal_probability)
    inter_l=np.zeros((len(data1), 16))
    post_l=np.zeros((len(data1), 16))
    
    print("Data ready for Vectorization")
    
    X=[title, description, parent_category_name, category_name, param_1, param123, image_top_1,
       cluster_5, cluster_10, cluster_20, user_type, activation_date, item_seq,
       numerical,image,location,price,prenum,postnum]
    Y=[output,inter_l,post_l]
    return (X,Y)

In [5]:
columns1=['title', 'description']
columns2=['parent_category_name','category_name', 'user_type','activation_date','item_seq_number',
         'param_1', 'param123','image_top_1','cluster_5', 'cluster_10','cluster_20','deal_probability']
columns3=['item_seq_count','item_seq_deal_avg','dis_days', 'edit_time', 'item_num','count',
          'avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
          'avg_pixel_width', 'width', 'height', 'size',
          'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2',
          'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2',
          'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2',
          'cus', 'ship', 'r_cus', 'r_ship', 'all_cus',
          'para_num',
          'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
          'title_num_words','title_num_unique_words','title_words_vs_unique',
          'description_num_words','description_num_unique_words','description_words_vs_unique',
          'price']
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
data1=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns1)
data2=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns2)
data3=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns3)


X_train, Y_train = get_keras_data(data1,data2,data3)


del data1,data2,data3
gc.collect()

Data ready for Vectorization


21

In [6]:
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
embedding_matrix1=np.load(path+'embedding_matrix1.npy')
gc.collect()


3

In [7]:
exp_decay = lambda init, fin, steps: (init/fin-1)/steps
#exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1

epochs = 16
batch_size = 4096
steps = ((len(Y_train[0]))/batch_size)*epochs
lr_init, lr_fin = 0.001, 0.00000597
lr_decay = exp_decay(lr_init, lr_fin, steps)
modelRNN = RNN_model()
K.set_value(modelRNN.optimizer.lr, lr_init)
K.set_value(modelRNN.optimizer.decay, lr_decay)

In [5]:
modelRNN.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title (InputLayer)              (None, 16)           0                                            
__________________________________________________________________________________________________
category_name (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
parent_category_name (InputLaye (None, 1)            0                                            
__________________________________________________________________________________________________
cluster_5 (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
cluster_10

In [ ]:
earlystop = EarlyStopping(monitor='loss', patience=10, verbose=1)

filepath='bestwhole.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint,earlystop]

hist = modelRNN.fit(X_train,Y_train, batch_size=batch_size, epochs=epochs,
                    verbose=1,callbacks=callbacks_list)





Epoch 1/16
1503424/1503424 [==============================] - 112s 74us/step - loss: 0.2690 - dense_8_loss: 0.2690 - dense_1_loss: 0.6585 - dense_2_loss: 0.3413 - dense_8_root_mean_squared_error: 0.2690 - dense_1_root_mean_squared_error: 0.6585 - dense_2_root_mean_squared_error: 0.3413

Epoch 00001: loss improved from inf to 0.26904, saving model to bestwhole.hdf5
Epoch 2/16
1503424/1503424 [==============================] - 110s 73us/step - loss: 0.2273 - dense_8_loss: 0.2273 - dense_1_loss: 0.6204 - dense_2_loss: 0.3153 - dense_8_root_mean_squared_error: 0.2273 - dense_1_root_mean_squared_error: 0.6204 - dense_2_root_mean_squared_error: 0.3153

Epoch 00002: loss improved from 0.26904 to 0.22728, saving model to bestwhole.hdf5
Epoch 3/16
1503424/1503424 [==============================] - 110s 73us/step - loss: 0.2250 - dense_8_loss: 0.2250 - dense_1_loss: 0.5884 - dense_2_loss: 0.2997 - dense_8_root_mean_squared_error: 0.2250 - dense_1_root_mean_squared_error: 0.5884 - dense_2_root_me

In [9]:
modelRNN.save('RNNmodel03.mod')

In [9]:
val_preds,~,~ = modelRNN.predict(X_test)

In [26]:
def rmse(y, y_pred):
    y=y.flatten()
    y_pred=y_pred.flatten()

    Rsum = np.sum((y - y_pred)**2)
    n = len(y)
    RMSE = np.sqrt(Rsum/n)
    return RMSE 

rmse(Y_test[0], val_preds)

In [18]:
model.save(filepath)
keras.models.load_model(filepath)

numpy.ndarray

In [21]:
(Y_test[0])

array([0.80323, 0.     , 0.10342, ..., 0.     , 0.     , 0.7376 ])

In [23]:
val_preds.flatten()


array([0.18857522, 0.06045391, 0.16316088, ..., 0.1368806 , 0.123833  ,
       0.39630845], dtype=float32)

In [12]:
exp_decay = lambda init, fin, steps: (init/fin-1)/steps
#exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1

epochs = 30
batch_size = 4096
steps = ((len(Y_train[0]))/batch_size)*epochs
lr_init, lr_fin = 0.005, 0.000005
lr_decay = exp_decay(lr_init, lr_fin, steps)
exp_decay(lr_init, lr_fin, steps)

0.11340515273887351

In [8]:
epochs = 30
batch_size = 4096
steps = ((len(Y_train[0]))/batch_size)*epochs

def exp_decay(epoch):
   initial_lrate = 0.005
   k = 0.1
   lrate = initial_lrate * exp(-k*t)
   return lrate

K.callbacks.LearningRateScheduler(schedule, verbose=0)

8809.12353515625

In [1]:
import sys
import platform
print("EXE:", sys.executable)  # you have to type the comma 
print("PATH is\n<%s>" % ">\n<".join(sys.path))
print(platform.architecture())

EXE: H:\anaconda\envs\tensorflow\python.exe
PATH is
<>
<H:\anaconda\envs\tensorflow\python35.zip>
<H:\anaconda\envs\tensorflow\DLLs>
<H:\anaconda\envs\tensorflow\lib>
<H:\anaconda\envs\tensorflow>
<H:\anaconda\envs\tensorflow\lib\site-packages>
<H:\anaconda\envs\tensorflow\lib\site-packages\IPython\extensions>
<C:\Users\lls\.ipython>
('64bit', 'WindowsPE')
